In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
#import total data
usmodels_df = pd.read_csv("us_energy_merged.csv")
usmodels_df.head()
usmodels_df.tail(5)

In [ ]:
#calulate excess between produced and consumed
usmodels_df["deficit"] = usmodels_df["production"]-usmodels_df["consumption"]
usmodels_df1 = usmodels_df.drop(["production", "consumption", "year","population", "price"], axis = 1)
usmodels_df1.head()

In [ ]:
#Creating lagged features for time series data
us_ts_df = pd.concat([usmodels_df1.shift(3), usmodels_df1.shift(2), usmodels_df1.shift(1), usmodels_df1], axis = 1)
us_ts_df.columns = ['deficit_t-3', 'deficit_t-2', 'deficit_t-1', "deficit_t"]
us_ts_df = us_ts_df.fillna(0)
us_ts_df[['production', 'population', 'price']] = usmodels_df[['production', 'population', 'price']]
us_ts_df.head()
# us_ts_df.tail(5)

In [ ]:
#creating linear regression model
y = us_ts_df["deficit_t"]
X = us_ts_df.drop('deficit_t', axis = 1)


In [ ]:
# train test and split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# create the linear regresson model
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
#show predictions
predictions = model.predict(X_test)
MSE = mean_squared_error(y_test, predictions)
r2 = model.score(X_test, y_test)
print(MSE, r2)


In [ ]:
#prediction for 2021
X_2021 = [-89652663.0, -88794414.0, -81248801.0, 9803617.86,336085.916,20.87]
y_2021 = model.predict([X_2021])
y_2021
print(f"the time series with lag features prediction for 2021 was {y_2021*1e-6} whereas simple linregress prediction was -95.01 quad")
print("The real deficit for 2021 was -85.17")

In [ ]:
#plotting for 2021 predictions
xp = ['with-lag-featues', 'no-lag-features', 'real']
yp = [-93, -95, -85]
plt.bar(xp,yp)

In [ ]:
# for 2021 total consumption was 97.33 quadrillion, total renewable 12.16 quadrillion, difference
# or deficit for 2021 = 